In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn import datasets
import pandas as pd


## Linear Regression Model

In [ ]:
class Linear_Regression(nn.Module):
  def __init__(self, n_inputs, n_outputs):
    super(Linear_Regression, self).__init__()
    self.regression_model = nn.Linear(n_features, n_outputs)
    
  def forward(self, x):
    y_pred = self.regression_model(x)
    return y_pred

model = Linear_Regression(n_features, 1)
learning_rate = 0.01
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
epochs=100

for epoch in range(epochs):
 
    y_pred = model(X_train)
  
      
    loss = criterion(y_pred, y_train)
  
      
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print('epoch:', {epoch+1}, 'loss:', loss.item())

 
  
  





Training Accuracy

In [ ]:
y_predicted = model(X_train)
y_predicted = y_predicted.round()

correct = 0
len_y = y_train.size(0)
for i in range(len_y):
  if y_train[i] == y_predicted[i]:
    correct+=1
print((correct/len_y) * 100)

93.62637362637362


Testing Accuracy

In [ ]:
y_predicted = model(X_test)
y_predicted = y_predicted.round()

correct = 0
len_y = y_test.size(0)
for i in range(len_y):
  if y_test[i] == y_predicted[i]:
    correct+=1
print((correct/len_y) * 100)

90.35087719298247


## Logistics Regression

In [ ]:
class Logistics_Regression(nn.Module):
    def __init__(self, n_input_features):
        super(Logistics_Regression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = Logistics_Regression(n_features)


num_epochs = 100
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


for epoch in range(num_epochs):
    
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    optimizer.zero_grad()

     # calculate the derivates of weights and biases
     # dw = (1/n_samples)*(np.dot(X.T, (pred - y)))
     # db = (1/n_samples)*(np.sum(pred-y))
    loss.backward()
    
    # self.weights = self.weights - self.lr*dw
    # self.bias = self.bias - self.lr*db
    optimizer.step()


    
    print('epoch:', {epoch+1}, 'loss:', loss.item())

y_predicted = model(X_test)
y_predicted = y_predicted.round()


Training accuracy

In [ ]:
y_predicted = model(X_train)
y_predicted = y_predicted.round()

correct = 0
len_y = y_train.size(0)
for i in range(len_y):
  if y_train[i] == y_predicted[i]:
    correct+=1
print((correct/len_y) * 100)

95.16483516483515


Test Accuracy

In [ ]:
y_predicted = model(X_test)
y_predicted = y_predicted.round()

correct = 0
len_y = y_test.size(0)
for i in range(len_y):
  if y_test[i] == y_predicted[i]:
    correct+=1
print((correct/len_y) * 100)

92.98245614035088


## Decision Tree

In [ ]:
class Decision_Tree(nn.Module):
    def __init__(self, n_input_features):
        super(Decision_Tree, self).__init__()
        self.decision_tree = nn.

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred



In [ ]:
import torch
import torch.nn as nn


class SDT(nn.Module):
    """Fast implementation of soft decision tree in PyTorch.
    Parameters
    ----------
    input_dim : int
      The number of input dimensions.
    output_dim : int
      The number of output dimensions. For example, for a multi-class
      classification problem with `K` classes, it is set to `K`.
    depth : int, default=5
      The depth of the soft decision tree. Since the soft decision tree is
      a full binary tree, setting `depth` to a large value will drastically
      increases the training and evaluating cost.
    lamda : float, default=1e-3
      The coefficient of the regularization term in the training loss. Please
      refer to the paper on the formulation of the regularization term.
    use_cuda : bool, default=False
      When set to `True`, use GPU to fit the model. Training a soft decision
      tree using CPU could be faster considering the inherent data forwarding
      process.
    Attributes
    ----------
    internal_node_num_ : int
      The number of internal nodes in the tree. Given the tree depth `d`, it
      equals to :math:`2^d - 1`.
    leaf_node_num_ : int
      The number of leaf nodes in the tree. Given the tree depth `d`, it equals
      to :math:`2^d`.
    penalty_list : list
      A list storing the layer-wise coefficients of the regularization term.
    inner_nodes : torch.nn.Sequential
      A container that simulates all internal nodes in the soft decision tree.
      The sigmoid activation function is concatenated to simulate the
      probabilistic routing mechanism.
    leaf_nodes : torch.nn.Linear
      A `nn.Linear` module that simulates all leaf nodes in the tree.
    """

    def __init__(
            self,
            input_dim,
            output_dim,
            depth=5,
            lamda=1e-3,
            use_cuda=False):
        super(SDT, self).__init__()

        self.input_dim = input_dim
        self.output_dim = output_dim

        self.depth = depth
        self.lamda = lamda
        self.device = torch.device("cuda" if use_cuda else "cpu")

        self._validate_parameters()

        self.internal_node_num_ = 2 ** self.depth - 1
        self.leaf_node_num_ = 2 ** self.depth

        # Different penalty coefficients for nodes in different layers
        self.penalty_list = [
            self.lamda * (2 ** (-depth)) for depth in range(0, self.depth)
        ]

        # Initialize internal nodes and leaf nodes, the input dimension on
        # internal nodes is added by 1, serving as the bias.
        self.inner_nodes = nn.Sequential(
            nn.Linear(self.input_dim + 1, self.internal_node_num_, bias=False),
            nn.Sigmoid(),
        )

        self.leaf_nodes = nn.Linear(self.leaf_node_num_,
                                    self.output_dim,
                                    bias=False)

    def forward(self, X, is_training_data=False):
        _mu, _penalty = self._forward(X)
        y_pred = self.leaf_nodes(_mu)

        # When `X` is the training data, the model also returns the penalty
        # to compute the training loss.
        if is_training_data:
            return y_pred, _penalty
        else:
            return y_pred

    def _forward(self, X):
        """Implementation on the data forwarding process."""

        batch_size = X.size()[0]
        X = self._data_augment(X)

        path_prob = self.inner_nodes(X)
        path_prob = torch.unsqueeze(path_prob, dim=2)
        path_prob = torch.cat((path_prob, 1 - path_prob), dim=2)

        _mu = X.data.new(batch_size, 1, 1).fill_(1.0)
        _penalty = torch.tensor(0.0).to(self.device)

        # Iterate through internal odes in each layer to compute the final path
        # probabilities and the regularization term.
        begin_idx = 0
        end_idx = 1

        for layer_idx in range(0, self.depth):
            _path_prob = path_prob[:, begin_idx:end_idx, :]

            # Extract internal nodes in the current layer to compute the
            # regularization term
            _penalty = _penalty + self._cal_penalty(layer_idx, _mu, _path_prob)
            _mu = _mu.view(batch_size, -1, 1).repeat(1, 1, 2)

            _mu = _mu * _path_prob  # update path probabilities

            begin_idx = end_idx
            end_idx = begin_idx + 2 ** (layer_idx + 1)

        mu = _mu.view(batch_size, self.leaf_node_num_)

        return mu, _penalty

    def _cal_penalty(self, layer_idx, _mu, _path_prob):
        """
        Compute the regularization term for internal nodes in different layers.
        """

        penalty = torch.tensor(0.0).to(self.device)

        batch_size = _mu.size()[0]
        _mu = _mu.view(batch_size, 2 ** layer_idx)
        _path_prob = _path_prob.view(batch_size, 2 ** (layer_idx + 1))

        for node in range(0, 2 ** (layer_idx + 1)):
            alpha = torch.sum(
                _path_prob[:, node] * _mu[:, node // 2], dim=0
            ) / torch.sum(_mu[:, node // 2], dim=0)

            coeff = self.penalty_list[layer_idx]

            penalty -= 0.5 * coeff * (torch.log(alpha) + torch.log(1 - alpha))

        return penalty

    def _data_augment(self, X):
        """Add a constant input `1` onto the front of each sample."""
        batch_size = X.size()[0]
        X = X.view(batch_size, -1)
        bias = torch.ones(batch_size, 1).to(self.device)
        X = torch.cat((bias, X), 1)

        return X

    def _validate_parameters(self):

        if not self.depth > 0:
            msg = ("The tree depth should be strictly positive, but got {}"
                   "instead.")
            raise ValueError(msg.format(self.depth))

        if not self.lamda >= 0:
            msg = (
                "The coefficient of the regularization term should not be"
                " negative, but got {} instead."
            )
            raise ValueError(msg.format(self.lamda))

## Breast Cancer Data

In [ ]:
breast_cancer_data = datasets.load_breast_cancer()
X, y = breast_cancer_data.data, breast_cancer_data.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [ ]:
diabetes = datasets.load_breast_cancer()
X, y = diabetes.data, diabetes.target

df = pd.DataFrame(diabetes,columns=diabetes.feature_names)
print(df.head())

In [ ]:
df=pd.DataFrame(data_diabetes.data,columns=data_diabetes.feature_names)


In [ ]:
df.shape   #569 rows, 30 columns

In [ ]:
df['Target']=data_diabetes.target

In [ ]:
df.head(30)